In [1]:
%%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms

In [2]:
from accelerate.utils import BnbQuantizationConfig
from accelerate import Accelerator
import transformers
import optimum
import bitsandbytes

In [3]:
import datetime
start_time = datetime.datetime.now()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

accelerator = Accelerator()

# Comment/Uncomment and use as per wish

# MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/2"
# MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/2"
# MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/13b-chat-hf/1"

# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

# model = model.to_bettertransformer()
model = accelerator.prepare(model)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [5]:
import pandas as pd
from tqdm import tqdm

DEBUG = False

TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE, nrows=NROWS, usecols=['id', 'rewrite_prompt'])

In [6]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_prompt(og_text, rewritten_text):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(og_text, 150)
    rewritten_text = truncate_txt(rewritten_text, 150)
    
    return f"""    
    Original Essay:
    \"""{og_text}\"""
    
    Rewritten Essay:
    \"""{rewritten_text}\"""
    
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
    You are trying to understand how the original essay was transformed into a new version.
    Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
    Start directly with the prompt, that's all I need. Output should be only line ONLY.
    """

In [7]:
import gc
import re

device = accelerator.device
tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

# https://www.kaggle.com/competitions/llm-prompt-recovery/discussion/481116
DEFAULT_TEXT = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."

res = []

while idx is not None:
    
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        res.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
        
    try:        
        messages = [
            {
                "role": "user",
                "content": gen_prompt(row["original_text"], row["rewritten_text"])
            }
        ]
        encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
        
        with torch.no_grad():
            encoded_output = model.generate(encoded_input, max_new_tokens=50, do_sample=True, pad_token_id=tokenizer.eos_token_id)
        
        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        decoded_output = result = re.sub(r"[\s\S]*\[\/INST\]", '', decoded_output, 1)
                
        res.append([row["id"], decoded_output])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]2024-03-10 07:23:44.107006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 07:23:44.107143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 07:23:44.373740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 1/1 [00:36<00:00, 36.28s/it]


In [8]:
# sub["rewrite_prompt"] = tdf['rewrite_prompt'].copy()
# sub.to_csv("submission.csv", index=False)
sub = pd.DataFrame(res, columns=['id', 'rewrite_prompt'])

sub.to_csv("sample_submission.csv", index=False)
sub.to_csv("submission.csv", index=False)

In [9]:
sub

,id,rewrite_prompt
0,9559194,"""Rewrite the original text as a humorous sea ..."


In [10]:
res

[[9559194,
  ' "Rewrite the original text as a humorous sea shanty, with each verse describing the competition and the task, and a chorus emphasizing the code competition aspect."']]